In [ ]:
# Install condacolab
!pip install -q condacolab
import condacolab
condacolab.install()

!conda create -n llava python=3.10 -y

!source activate llava

In [ ]:
!git clone https://github.com/Czi24/LLaVA-20240713.git
%cd /content/LLaVA-20240713/
!pip install -e .

In [15]:
!pip install huggingface_hub protobuf
%cd /content
from huggingface_hub import snapshot_download
snapshot_download(repo_id="liuhaotian/llava-v1.6-vicuna-7b", local_dir="llava-v1.6-vicuna-7b")

/content


Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

model.safetensors.index.json:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/936 [00:00<?, ?B/s]

trainer_state.json:   0%|          | 0.00/719k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.68k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/7.04k [00:00<?, ?B/s]

'/content/llava-v1.6-vicuna-7b'

In [20]:
import threading
import subprocess

# start Controller
threading.Thread(
    target=lambda: subprocess.run(
        ['python3', '-m', 'llava.serve.controller', '--host', '0.0.0.0', '--port', '10000'],
        check=True
    ),
    daemon=True
).start()

In [21]:
import threading
import subprocess

# Define the command to run the model worker
command = [
    'python3', '-m', 'llava.serve.model_worker',
    '--host', '0.0.0.0',
    '--controller', 'http://localhost:10000',
    '--port', '40000',
    '--worker', 'http://localhost:40000',
    '--model-path', '/content/llava-v1.6-vicuna-7b',
    '--load-8bit'
]

# Run the command in a separate thread
thread = threading.Thread(
    target=lambda: subprocess.run(command, check=True, shell=False),
    daemon=True
)
thread.start()


In [ ]:
!python3 -m llava.serve.gradio_web_server --controller http://localhost:10000 --model-list-mode reload --share